In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("../data/raw/Olympic_Athlete_Biography.csv")
df.head()

,athlete_id,name,sex,born,height,weight,country,country_noc,description,special_notes
0,65649,Ivanka Bonova,Female,4 April 1949,166.0,55,Bulgaria,BUL,Personal Best: 400 – 53.54 (1980).,NaN
1,112510,Nataliya Uryadova,Female,15 March 1977,184.0,70,Russian Federation,RUS,NaN,Listed in Olympians Who Won a Medal at the Eur...
2,114973,Essa Ismail Rashed,Male,14 December 1986,165.0,55,Qatar,QAT,Personal Best: 10000 – 27:20.97 (2006).,Listed in Olympians Who Won a Medal at the Asi...
3,30359,Péter Boros,Male,12 January 1908,NaN,NaN,Hungary,HUN,"Between 1927 and 1938, Péter Boros competed as...",NaN
4,50557,Rudolf Piowatý,Male,28 April 1900,NaN,NaN,Czechoslovakia,TCH,Rudolf Piowaty joined the Czechoslovak militar...,NaN


In [3]:
def process_born(row):

    if pd.isna(row):
        return row
    else:
        if row[0] == '(':
            row = row[1:-1]
        row_splitted = row.strip().split()
        return row_splitted[-1]


# Only take year 
df['born'] = df['born'].apply(process_born)

df['born']

print(type(df['country_noc'].unique()))


<class 'numpy.ndarray'>


In [4]:
print(df[df['weight'] == 77])

Empty DataFrame
Columns: [athlete_id, name, sex, born, height, weight, country, country_noc, description, special_notes]
Index: []


In [5]:
# if weight is within approx => take average 
height_male_average = 176.1
height_female_average = 163.1
weight_male_average = 77
weight_female_average = 59

def weight_process(row):
  row = str(row)
  splitted_row = row.split('-')
  if len(splitted_row) > 1:
      result = (float(splitted_row[0]) + float(splitted_row[1])) / 2
      return result

df['weight'] = df['weight'].apply(weight_process)

index = df['country_noc'].unique()
columns = pd.MultiIndex.from_tuples([
    ('weight', 'Male'),
    ('weight', 'Female'),
    ('height', 'Male'),
    ('height', 'Female')
])
new_df = pd.DataFrame(index=index, columns= columns)

for country_noc in index:
   new_df.loc[country_noc, ('weight', 'Male')] = df[(df['country_noc'] == country_noc) & (pd.notna(df['weight']) & (df['sex'] == 'Male'))]['weight'].mean()
   new_df.loc[country_noc, ('weight', 'Female')] = df[(df['country_noc'] == country_noc) & (pd.notna(df['weight']) & (df['sex'] == 'Female'))]['weight'].mean()
   new_df.loc[country_noc, ('height', 'Male')] = df[(df['country_noc'] == country_noc) & (pd.notna(df['height']) & (df['sex'] == 'Male'))]['height'].mean()
   new_df.loc[country_noc, ('height', 'Female')]= df[(df['country_noc'] == country_noc) & (pd.notna(df['height']) & (df['sex'] == 'Female'))]['height'].mean()

# print(df[(pd.isna(df['weight']) & (df['sex'] == 'Male'))]['country_noc'])
for index, row in df.iterrows():
   if pd.isna(row['weight']):
      mean_weight = new_df.loc[row['country_noc'], ('weight', row['sex'])]
      if pd.notna(mean_weight):
         df.loc[index, 'weight'] = mean_weight
      else:
         df.loc[index, 'weight'] = weight_male_average if row['sex']=='Male' else weight_female_average

   if pd.isna(row['height']):
      mean_height = new_df.loc[row['country_noc'], ('height', row['sex'])]
      if pd.notna(mean_weight):
         df.loc[index, 'height'] = mean_height
      else:
         df.loc[index, 'height'] = height_male_average if row['sex']=='Male' else height_female_average


#if na set value 0
# df['height'] = df['height'].apply(lambda x : x if pd.notna(x) else 0)
# df['weight'] = df['weight'].apply(lambda x : x if pd.notna(x) else 176)


df.fillna("No information !", inplace = True)

# Thay thế NaN bằng 0


In [6]:
# print(df[(df['height'] == height_female_average)]['country_noc'])
# print(new_df)
df.head()

,athlete_id,name,sex,born,height,weight,country,country_noc,description,special_notes
0,65649,Ivanka Bonova,Female,1949,166.000000,59.000000,Bulgaria,BUL,Personal Best: 400 – 53.54 (1980).,No information !
1,112510,Nataliya Uryadova,Female,1977,184.000000,88.500000,Russian Federation,RUS,No information !,Listed in Olympians Who Won a Medal at the Eur...
2,114973,Essa Ismail Rashed,Male,1986,165.000000,104.500000,Qatar,QAT,Personal Best: 10000 – 27:20.97 (2006).,Listed in Olympians Who Won a Medal at the Asi...
3,30359,Péter Boros,Male,1908,180.330201,84.285714,Hungary,HUN,"Between 1927 and 1938, Péter Boros competed as...",No information !
4,50557,Rudolf Piowatý,Male,1900,180.390011,161.000000,Czechoslovakia,TCH,Rudolf Piowaty joined the Czechoslovak militar...,No information !


In [7]:
df.to_csv('../data/processed/Olympic_Athlete_Biography.csv', index=False)